In [1]:
import numpy as np
import h5py
import sys

sys.path.append("../src")
from Load_data.prepare_data import *

In [2]:
def build_binary_any_dataset(
    cache_path,
    train_image_dir,
    test_image_dir,
    train_xml,
    test_xml,
    img_size=(64, 64)
):
    os.makedirs(os.path.dirname(cache_path), exist_ok=True)

    print("🛠️ Building BINARY_ANY dataset")

    # -------- TRAIN --------
    X_train, fn_train = load_and_resize_images(train_image_dir, img_size)
    raw_train = parse_cvat_xml_all_labels(train_xml)
    labels_train = label_Y_binary(raw_train)
    Y_train = build_label_array(fn_train, labels_train)

    # -------- TEST --------
    X_test, fn_test = load_and_resize_images(test_image_dir, img_size)
    raw_test = parse_cvat_xml_all_labels(test_xml)
    labels_test = label_Y_binary(raw_test)
    Y_test = build_label_array(fn_test, labels_test)

    with h5py.File(cache_path, "w") as f:
        f.create_dataset("X_train", data=X_train, compression="gzip")
        f.create_dataset("Y_train", data=Y_train, compression="gzip")
        f.create_dataset("X_test",  data=X_test,  compression="gzip")
        f.create_dataset("Y_test",  data=Y_test,  compression="gzip")

    print(f"✅ Dataset built and saved to {cache_path}")


In [3]:
build_binary_any_dataset(
    cache_path="../EIDSeg_Dataset/cache/eidseg_64x64_binary_any.h5",
    train_image_dir="../EIDSeg_Dataset/data/train/images/default",
    test_image_dir ="../EIDSeg_Dataset/data/test/images/default",
    train_xml="../EIDSeg_Dataset/data/train/train.xml",
    test_xml ="../EIDSeg_Dataset/data/test/test.xml",
    img_size=(64, 64)
)




🛠️ Building BINARY_ANY dataset
Final X shape: (2612, 64, 64, 3)
Final X shape: (327, 64, 64, 3)
✅ Dataset built and saved to ../EIDSeg_Dataset/cache/eidseg_64x64_binary_any.h5


In [4]:
def build_destroyed_coverage_dataset(
    cache_path,
    train_image_dir,
    test_image_dir,
    train_xml,
    test_xml,
    img_size=(64, 64),
    min_coverage=0.3
):
    os.makedirs(os.path.dirname(cache_path), exist_ok=True)

    print("🛠️ Building DESTROYED_COVERAGE dataset")

    # -------- TRAIN --------
    X_train, fn_train = load_and_resize_images(train_image_dir, img_size)
    labels_train = parse_destroyed_with_size_check(
        train_xml, min_coverage=min_coverage
    )
    Y_train = build_label_array(fn_train, labels_train)

    # -------- TEST --------
    X_test, fn_test = load_and_resize_images(test_image_dir, img_size)
    labels_test = parse_destroyed_with_size_check(
        test_xml, min_coverage=min_coverage
    )
    Y_test = build_label_array(fn_test, labels_test)

    with h5py.File(cache_path, "w") as f:
        f.create_dataset("X_train", data=X_train, compression="gzip")
        f.create_dataset("Y_train", data=Y_train, compression="gzip")
        f.create_dataset("X_test",  data=X_test,  compression="gzip")
        f.create_dataset("Y_test",  data=Y_test,  compression="gzip")

    print(f"✅ Dataset built and saved to {cache_path}")


In [5]:

build_destroyed_coverage_dataset(
    cache_path="../EIDSeg_Dataset/cache//eidseg_64x64_destroyed_cov30.h5",
    train_image_dir="../EIDSeg_Dataset/data/train/images/default",
    test_image_dir ="../EIDSeg_Dataset/data/test/images/default",
    train_xml="../EIDSeg_Dataset/data/train/train.xml",
    test_xml ="../EIDSeg_Dataset/data/test/test.xml",
    img_size=(64, 64),
    min_coverage=0.3
)


🛠️ Building DESTROYED_COVERAGE dataset
Final X shape: (2612, 64, 64, 3)
Final X shape: (327, 64, 64, 3)
✅ Dataset built and saved to ../EIDSeg_Dataset/cache//eidseg_64x64_destroyed_cov30.h5


In [8]:
def build_flattened_cache(
    source_cache_path,
    flat_cache_path
):
    os.makedirs(os.path.dirname(flat_cache_path), exist_ok=True)

    print("🛠️ Building FLATTENED cache")

    with h5py.File(source_cache_path, "r") as src:
        X_train = src["X_train"][:]
        Y_train = src["Y_train"][:]
        X_test  = src["X_test"][:]
        Y_test  = src["Y_test"][:]

    # flatten + normalize
    train_x_flat = X_train.reshape(X_train.shape[0], -1).T / 255.0
    test_x_flat  = X_test.reshape(X_test.shape[0],  -1).T / 255.0

    with h5py.File(flat_cache_path, "w") as f:
        f.create_dataset("train_x", data=train_x_flat, compression="gzip")
        f.create_dataset("train_y", data=Y_train, compression="gzip")
        f.create_dataset("test_x",  data=test_x_flat,  compression="gzip")
        f.create_dataset("test_y",  data=Y_test,  compression="gzip")

    print(f"✅ Flattened cache saved to {flat_cache_path}")


In [10]:
build_flattened_cache(
    source_cache_path="../EIDSeg_Dataset/cache//eidseg_64x64_binary_any.h5",
    flat_cache_path="../EIDSeg_Dataset/cache//eidseg_64x64_binary_any_flat.h5"
)


🛠️ Building FLATTENED cache
✅ Flattened cache saved to ../EIDSeg_Dataset/cache//eidseg_64x64_binary_any_flat.h5


In [11]:
build_flattened_cache(
    source_cache_path="../EIDSeg_Dataset/cache//eidseg_64x64_destroyed_cov30.h5",
    flat_cache_path="../EIDSeg_Dataset/cache//eidseg_64x64_destroyed_cov30_flat.h5"
)


🛠️ Building FLATTENED cache
✅ Flattened cache saved to ../EIDSeg_Dataset/cache//eidseg_64x64_destroyed_cov30_flat.h5
